In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as data
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
import random

use_cuda = True
device_id = 3
from tensorflow.contrib.keras.python.keras.datasets.imdb import load_data, get_word_index

max_features = 5000
batch_size = 32
epochs = 15
learning_rate = 0.001

In [2]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [3]:
(x_train, y_train), (x_test, y_test) = load_data(num_words=max_features)


In [4]:
def pad(tensor, length):
    return torch.cat([tensor, tensor.new(length - tensor.size(0),*tensor.size()[1:]).zero_()])

In [5]:
def sortedText(idx, xs, ys):
    batch_xs = xs[idx]
    batch_ys = ys[idx]
    lengths = np.array([len(x) for x in batch_xs])
    sort_idx = np.argsort(lengths)[::-1]
    return batch_xs[sort_idx], lengths[sort_idx], batch_ys[sort_idx]


def textTensor(idx, xs, ys):
    batch_xs, lengths, batch_ys = sortedText(idx, xs, ys)
    max_length = lengths[0]
    return torch.cat([pad(torch.Tensor(x), max_length).view(max_length, 1)
                      for x in batch_xs], 1).long(), list(lengths), torch.FloatTensor(batch_ys)


In [12]:
class IMDB_Classifier(nn.Module):
    def __init__(self, input_size, embedding_size, filters, hidden_size):
        super(IMDB_Classifier, self).__init__()
        self.embedding_size = embedding_size
        self.filters = filters
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(0.2)
        self.conv = nn.Conv1d(embedding_size, filters, 3, stride=1, padding=0)
        self.dense1 = nn.Linear(filters, hidden_size)
        self.dense2 = nn.Linear(hidden_size, 1)
        
        
    def forward(self, word_input):
        word_embedded = self.embedding(word_input)
        output = self.dropout(word_embedded)
        output = output.transpose(0, 1).transpose(1,2)
        output = F.relu(self.conv(output))
        output = torch.max(output, 2)[0].squeeze()
        output = F.relu(self.dropout(self.dense1(output)))
        output = F.sigmoid(self.dense2(output))
        
        return output
        

In [16]:


clf = IMDB_Classifier(max_features, 50, 250, 250)
clf = clf.cuda(device_id) if use_cuda else clf
optimizer = optim.Adam(clf.parameters(), lr=0.001)
criterion = nn.BCELoss()

start = time.time()
for epoch in range(1, 101):
    losses = 0
    indices = np.random.permutation(np.array(range(25000)))
    for i in range(1, indices.shape[0] / 32 + 1):
        x, lengths, y = textTensor(indices[(i-1)*32:i*32], x_train, y_train)
        x, y = Variable(x), Variable(y)

        (x, y) = (x.cuda(device_id), y.cuda(device_id)) if use_cuda else (x, y)

        output = clf(x)
        loss = criterion(output, y)
        losses += loss.data[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print("batch: {}, batch_loss: {}".format(i, losses / i))

    print("Epoch: {}, time: {}, loss: {}".format(epoch, timeSince(start, float(epoch) / epochs), losses/(i)))



    total = 0
    correct = 0
    for i in range(250):
        x, lengths, y = textTensor(range(25000)[i*100:(i+1)*100], x_train, y_train)
        #x, lengths, y = textTensor(torch.LongTensor(range(32)), x_train, y_train)
        x, y = Variable(x, volatile=True), Variable(y)
        (x, y) = (x.cuda(device_id), y.cuda(device_id)) if use_cuda else (x, y)
        output = clf(x)
        output = output > 0.5
        correct += (output.float() == y).sum().data[0]
        total += y.size(0)
    print('Accuracy of the network on the {} texts: {} %'.format(y_test.shape[0], 100. * correct / total))

batch: 10, batch_loss: 0.795276343822
batch: 20, batch_loss: 0.751570284367
batch: 30, batch_loss: 0.739358232419
batch: 40, batch_loss: 0.729999986291
batch: 50, batch_loss: 0.722238819599
batch: 60, batch_loss: 0.719582094749
batch: 70, batch_loss: 0.718231364659
batch: 80, batch_loss: 0.715972481668
batch: 90, batch_loss: 0.713740558757
batch: 100, batch_loss: 0.711170747876
batch: 110, batch_loss: 0.709835349972
batch: 120, batch_loss: 0.708382978539
batch: 130, batch_loss: 0.706178793999
batch: 140, batch_loss: 0.705328278031
batch: 150, batch_loss: 0.704142908255
batch: 160, batch_loss: 0.702794122323
batch: 170, batch_loss: 0.700931138501
batch: 180, batch_loss: 0.700097064177
batch: 190, batch_loss: 0.699008051032
batch: 200, batch_loss: 0.697991194725
batch: 210, batch_loss: 0.69692274474
batch: 220, batch_loss: 0.695957202532
batch: 230, batch_loss: 0.694566332257
batch: 240, batch_loss: 0.693408409506
batch: 250, batch_loss: 0.691588876247
batch: 260, batch_loss: 0.689360033

KeyboardInterrupt: 